<a href="https://colab.research.google.com/github/shazoop/EE290/blob/master/Spectrum_Sensing1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectrum Sensing (Sparsity)

## Looking Ahead

In this series of examples, we are going to explore the use of compressive sensing for spectrum sensing in a simulated frequency-multiplexed channel sharing scheme. This first notebook will facilitate our exploration of the spectral content of simulated data from such a scheme, and expose the underlying sparsity of the frequency spectrum, which will make compressive sensing techniques applicable in later notebooks.

The precise nature of the model frequency-multiplexed transmission scheme is described in the homework. Here, we will load the simulated data from a file, remind ourselves of some points germane to working with RF signals, and then finally examine the frequency sparsity of the data.

## Downloading the Data

We download the data file from my Github page. The file is `ss_signal.mat`.

In [0]:
!git clone https://github.com/hwagyesa/ss_data.git

## Loading and Understanding the Data

Next, we load the signal and the provided parameters into memory, and look at the available keys.

In [0]:
import scipy.io as sio
import numpy as np
import bokeh.plotting as bpl
import pprint

D = sio.loadmat('/content/ss_data/ss_signal.mat', squeeze_me=True)
D_short = {k: v for k, v in D.items() if not k.startswith('_')}
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(D_short)

Here is a description of the contained parameters:
- `x`: The frequency-multiplexed channel's voltage signal. All subsequent parameters relate to different aspects of this signal.
- `SNR`: The signal-to-noise ratio of `x`, measured in decibels. Here we again emphasize that `x` is a voltage signal, and that we have added i.i.d. Gaussian random variables to each sample bin to ensure the specified SNR is met.
- `fmin`: The minimum frequency in the channel's allocated frequency band, measured in Hz.
- `B`: The bandwidth of a single user's subband allocation (in frequency), measured in Hz. 
- `Nuser`: The number of users, or subbands, in the allocated frequency range. We have the relation `fmax - fmin = Nuser * B`, so we can derive the maximum band frequency `fmax` from the previous three parameters.
- `fs`: The sampling frequency of the signal `x`, in Hz. Here we have used Nyquist-rate sampling, so `fs = 2 * fmax`.
- `Nslots`: This is a time-transmission parameter, which tells us how many subsequent transmission slots the signal contains.
- `slot_time`: Another time parameter that tells us the duration of a single transmission slot, measured in seconds. The total number of samples in `x` is thus equal to `fs * slot_time * Nslots`, and the total time duration of the signal is `slot_time * Nslots`.

Interpreting given the values above, we see that (for example) the slot bandwidth of our system is 100 kHz and the slot time is 1 ms, among other things.

### Derived Parameters

We compute some of the derived parameters mentioned above in the code block below.


In [0]:
## Move dictionary vars into locals
B = D['B']
Nslots = D['Nslots']
Nuser = D['Nuser']
SNR = D['SNR']
fmin = D['fmin']
fs = D['fs']
slot_time = D['slot_time']
x = D['x']

## Derived parameters
fmax = Nuser * B + fmin
T = slot_time * Nslots
Nsamp = len(x)
slot_dur = Nsamp // Nslots

## Visualizing the Data

Now that we have specified the meaning of the different signal parameters, we can plot the frequency content of different transmission intervals of `x`. First, we generate frequency and time vectors which have the proper physical interpretation in light of our parameter specifications above.

In particular, our signal is sampled at the Nyquist rate, and so its (unrealizable) DTFT is `fs`-periodic, with no aliasing. This tells us what units to place on the frequency axis after we FFT the signal `x`. The time axis is similarly straightforward to determine.



In [0]:
## Time and frequency axes
f_axis = fs * np.arange(-Nsamp/2+1, Nsamp/2+1)/Nsamp
t_axis = Nslots * slot_time * np.arange(0, Nsamp)/Nsamp
f_axis = np.squeeze(f_axis)
t_axis = np.squeeze(t_axis)

In [0]:
## Plot the time signal, over a single slot time
bpl.output_notebook()
h = bpl.figure(title="Time-domain signal x")
h.xaxis.axis_label = 'Time (ms)'
h.yaxis.axis_label = 'Voltage (V)'
h.line(t_axis[0:slot_dur]*1.0e3, x[0:slot_dur])
bpl.show(h)

This plot does not tell us much about the characteristics of the signal `x`, which is of course more naturally viewed in the frequency domain. We plot this next.

**Note**: We use some tricks with the Fourier transform below, which you should make sure you understand. *First*, we use `fftshift` after calling `fft`: this is because the returned signal from `fft` represents the frequency content in the interval `[0, fs]`, whereas we are more used to having a DC-centered view of the frequency content, from `[-fs/2, fs/2]`. `fftshift` performs this adjustment by shuffling the array. *Second*, we also downsample the `f_axis` variable: this is because when we look at the frequency content in the first transmission slot, we have a shorter time duration to work with, so although the overall frequency band that we view is the same (we did not change the sampling frequency!), the frequency resolution becomes worse.

In [0]:
slot1_x = x[0:slot_dur]
slot1_X = np.fft.fftshift(np.fft.fft(slot1_x))
## We need to downsample the frequency axis to match the shorter slot time
slot_f_axis = f_axis[::Nslots]

## Make the plot
bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during first slot")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis * 1e-6, 20 * np.log10(np.absolute(slot1_X)))
bpl.show(h)


We see that the signal consists of just a few different transmitters during the first transmission slot. We can get a closer view by plotting just our frequency band of interest, over the positive frequency axis.

In [0]:
inband_mask = np.where((slot_f_axis >= fmin) & (slot_f_axis <= fmax))[0]

bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during first slot (positive band-of-interest)")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis[inband_mask] * 1e-6,
       20 * np.log10(np.absolute(slot1_X[inband_mask])))
bpl.show(h)

In [0]:
slot2_x = x[slot_dur:2*slot_dur]
slot2_X = np.fft.fftshift(np.fft.fft(slot2_x))
## We need to downsample the frequency axis to match the shorter slot time
## Make the plot
bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during second slot")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis * 1e-6, 20 * np.log10(np.absolute(slot2_X)))
bpl.show(h)


Finally, we can get an idea of which users are transmitting by zooming into a single subband, given our knowledge of the transmit scheme parameters. We know that the bands have bandwidth `B`, and that the frequency interval `[fmin, fmax]` is divided up into `Nuser` of them. From here we can compute the center frequency for each of the `Nuser` bands in the interval, and view their content by looking `B/2` above and below. We do this below, also viewing some of the nearby bands.

In [0]:
f_center_230 = 23.55 * 1e6
subband230_mask = np.where((slot_f_axis >= f_center_230-4*B)
                           & (slot_f_axis <= f_center_230+4*B))[0]

bpl.output_notebook()
h = bpl.figure(title="Frequency-domain signal x, during first slot (around 230th subband)")
h.xaxis.axis_label = 'Frequency (MHz)'
h.yaxis.axis_label = 'Power (dB)'
h.line(slot_f_axis[subband230_mask] * 1e-6,
       20 * np.log10(np.absolute(slot1_X[subband230_mask])))
bpl.show(h)

We see that the 230th subband, centered at frequency 23.55 MHz, is occupied during the first transmission slot: its total frequency band is `[23.5, 23.6]` MHz, and although there is some leakage into the neighboring bands, the leakage is small (about 10 dB down from the transmission power, it appears from the figure).

## Your Tasks

Each level three header below contains a task you should complete. See the homework handout for additional details.

![alt text](https://)### Task 1: Evaluating Subband Occupancy Over Time

The goal of this task is to generate a discrete analog of the time signal `x` that allows us to assess the subband-level sparsity of `x` during each time slot. Perform the following tasks:
1. Given the frequency content of a single time slot of `x`, write code to output a vector with `Nuser` entries: the `i`-th entry is 1 if the corresponding subband is occupied during this time slot and 0 otherwise. For example, you can expect as input the fft of a single time slot as we generated for the first timeslot in one of the cells above.
2. Apply the code you wrote in the previous step to estimate the length-`Nuser` vector of subband occupancies for each of the `Nslots` time slots contained in the signal `x`.

For this part of the problem, your **output** should be the `Nuser`-by-`Nslots` matrix of subband occupancies for the signal `x`.

**Hint**: You can use the fact that the SNR of the signal `x` is quite high to easily test whether a given subband is occupied or not: index it properly as we have done above (after computing the center frequencies of each subband), and apply something like a simple integrate-and-threshold test.


In [0]:
#Task 1: Generating the occupancy matrix

sparSig = np.empty((Nuser, Nslots))

'''
Since each user occupies 1/Nuser of the total frequency band, just split DFT into Nuser frequency intervals.
Base on the above graphs, usage in a user's band corresponds to there being a power spike above 0.
'''

for i in range(Nslots):
  slot = x[i*slot_dur:(i+1)*slot_dur]
  slot_x = np.fft.fftshift(np.fft.fft(slot))
  power_x = 20 * np.log10(np.absolute(slot_x))
  B2 = len(slot_x) // Nuser
  for j in range(Nuser):
        if np.max(power_x[j*B2 : (j+1)*B2]) > 0:
          sparSig[j,i] = 1
        else:
          sparSig[j,i] = 0

### Task 2: Basic Questions about the Sparsity-Over-Time of the Signal

Provide answers to the following questions regarding interpretation of your subband occupancies signal generated in the previous task.
1. What is the maximum density of the signal `x` within a single slot over time? Here, density means "number of users transmitting simultaneously in a single time slot, divided by total number of users", and can be related to the sparsity of your zero-one matrix from the previous task. What about the average density?
2. Is the spectral occupancy of a single subband as a function of slot time a sparse signal? What about the spectral occupancy of the total `[fmin, fmax]` frequency band within a single time slot? Explain.


In [0]:
#Question 1

densitySig = np.sum(sparSig, axis = 0)/Nuser
maxDensity = np.amax(densitySig)
avgDensity = np.sum(densitySig)/Nslots
print("max density: %s ; average density %s" % (maxDensity, avgDensity))

#Max density is .25, while average density is .2

In [0]:
#Question 2
#For each user's subband, form an Nslot-long vector where each entry is 1/0 depending on occupancy in that time slot.

#bandOcc is sum, over time, of occupancy of that band
bandOcc = np.sum(sparSig, axis = 1)
print("max occ: %s ; average occ %s" % (np.max(bandOcc), np.sum(bandOcc)/Nuser))

#On average, a user is transmitting 3 out of the 16 time slots. 

bandOcc

#Looking at number of occupied slots per band, look to be pretty sparse.

In [0]:
'''
As for the spectral occupancy of the total spectrum within a time slot, this is proportional to density of the signal in part 1. Since we already saw that
the density is generally around 20%, this means that the total spectrum occupancy is quite sparse per time slot.
'''